In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
# Load pre-trained ResNet50 model for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 42s 0us/step 


In [3]:
def load_images_from_folder(folder):
    images = []
    labels = []
    class_labels = os.listdir(folder)
    
    for label in class_labels:
        class_path = os.path.join(folder, label)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (224, 224))  # Resize to match ResNet50 input
                    img = img_to_array(img)
                    img = preprocess_input(img)
                    images.append(img)
                    labels.append(label)
    
    return np.array(images), np.array(labels)

In [4]:
# Load training data
train_path = 'PlantDoc-Dataset/train'
X_train, y_train = load_images_from_folder(train_path)

# Load testing data
test_path = 'PlantDoc-Dataset/test'
X_test, y_test = load_images_from_folder(test_path)


In [6]:
# Extract features
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

73/73 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step


In [7]:
# Standardize features
scaler = StandardScaler()
X_train_features = scaler.fit_transform(X_train_features)
X_test_features = scaler.transform(X_test_features)

In [8]:
# Train and evaluate SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_features, y_train_encoded)
y_pred_svm = svm_model.predict(X_test_features)
svm_accuracy = accuracy_score(y_test_encoded, y_pred_svm)
print(f'SVM Accuracy: {svm_accuracy:.2f}')

SVM Accuracy: 0.54


In [9]:
# Train and evaluate XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_features, y_train_encoded)
y_pred_xgb = xgb_model.predict(X_test_features)
xgb_accuracy = accuracy_score(y_test_encoded, y_pred_xgb)
print(f'XGBoost Accuracy: {xgb_accuracy:.4f}')

C:\Users\shikh\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:32:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.4788


## Not Working